In [ ]:
!pip install flask flask_cors flask_ngrok

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2024-02-18 18:33:15--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 18.205.222.128, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  18.1MB/s    in 0.7s    

2024-02-18 18:33:16 (18.1 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
!pip install pyngrok
from pyngrok import ngrok

# Enter your ngrok authentication token here
ngrok.set_auth_token("2cRnHQdmuO04eM0YVgAj9aDmz2P_6B5XpjPTvVQKAAfk7GbBi")

# Start ngrok tunnel
ngrok_tunnel = ngrok.connect(5000)

# Print ngrok URL
print("Ngrok Tunnel URL:", ngrok_tunnel.public_url)


Ngrok Tunnel URL: https://4d79-34-125-18-162.ngrok-free.app


In [ ]:
from flask import Flask, jsonify, request
from flask_cors import CORS
from flask_ngrok import run_with_ngrok
from transformers import pipeline

# Initialize the Flask application
application = Flask(__name__)

# Run the Flask application with ngrok
run_with_ngrok(application)

# Enabling CORS for all routes
CORS(application)

# Random API key
API_KEY = 'ehtisham'

# Load the text generation pipeline
pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0")

# Home Page Route
@application.route('/', methods=['GET'])
def home():
    return "Home route"

# Generate Response Route
@application.route('/generate_response', methods=['POST'])
def generate_response():
    try:
        if 'api_key' not in request.json:
            return jsonify({'error': 'API key not passed by the user.'}), 401

        if request.json['api_key'] != API_KEY:
            return jsonify({'error':'API key not matched.'}), 401

        # Get the input history from the request
        history = request.json.get('history')

        # Check if history is None or empty
        if history is None or len(history) == 0:
            return jsonify({'error': 'Please provide a valid history'}), 400

        # Check if the last message in history has the role of the user
        if history[-1]['role'] != 'user':
            return jsonify({'error': 'The last message in history should be from the user.'}), 400

        try:
            # Concatenate the history messages into a single string
            history_text = " ".join([msg['message'] for msg in history])

            # Generate response using the text generation pipeline
            response = pipe(history_text, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)

            # Extract the generated text from the response
            generated_text = response[0]["generated_text"]

            # Append the generated text to the history with assistant role
            history.append({
                'role': 'assistant',
                'message': generated_text
            })

            # Construct the JSON response manually
            response_data = {'history': history}
            return jsonify(response_data), 200

        except Exception as e:
            return jsonify({'error': str(e)}), 500

    except Exception as e:
        return jsonify({'error': str(e)}), 500

# Run the Flask application
if __name__ == '__main__':
    application.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://4d79-34-125-18-162.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [18/Feb/2024 18:34:32] "POST /generate_response HTTP/1.1" 400 -
INFO:werkzeug:127.0.0.1 - - [18/Feb/2024 18:36:25] "POST /generate_response HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Feb/2024 18:45:12] "POST /generate_response HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Feb/2024 18:47:32] "POST /generate_response HTTP/1.1" 200 -
